In [1]:
import geemap
import ee
ee.Initialize()
reducer = ee.Reducer.mean()
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [6]:
'''添加矢量数据'''

duchang_shp = "D:\水稻有关数据\都昌矢量数据\都昌矢量数据.shp"
duchang = geemap.shp_to_ee(duchang_shp)
Map.addLayer(duchang, {}, 'duchang')
Map.centerObject(duchang, zoom=10)
roi = duchang.geometry()  #获取矢量的范围

In [7]:
def clipp(image):
     return image.clip(roi)

In [8]:
'''添加影像集'''

evi_collections = ee.ImageCollection("LANDSAT/LE07/C01/T1_8DAY_EVI").filterDate('2000-01-01','2020-12-31').filterBounds(roi).map(clipp)

mndwi_collections = ee.ImageCollection("LANDSAT/LE07/C01/T1_8DAY_NDWI").filterDate('2000-01-01','2020-12-31').filterBounds(roi).map(clipp)

In [9]:
#利用日期差异为SG拟合添加预测因子
#我们准备3个订单的配合，但是以后可以适应较低订单的配合
def func_way(img):
    dstamp = ee.Date(img.get('system:time_start'))
    ddiff = dstamp.difference(ee.Date('2000-01-01'), 'hour')
    img = img.select(['EVI']).set('date', dstamp)
    return img.addBands(ee.Image(1).toFloat().rename('constant')).\
    addBands(ee.Image(ddiff).toFloat().rename('t')).\
    addBands(ee.Image(ddiff).pow(ee.Image(2)).toFloat().rename('t2')).\
    addBands(ee.Image(ddiff).pow(ee.Image(3)).toFloat().rename('t3'))

evi_res = evi_collections.map(func_way)

In [10]:
window_size = 9
half_window = (window_size - 1)/2
imageAxis = 0
bandAxis = 1
order = 3
coeffFlattener = [['constant', 'x', 'x2', 'x3']]
indepSelectors = ['constant', 't', 't2', 't3']

In [11]:
array = evi_res.toArray()

In [12]:
def getLocalFit(i):
    #获取与SG平滑器的window_size相对应的切片
    subarray = array.arraySlice(imageAxis, ee.Number(i).int(), ee.Number(i).add(window_size).int())
    predictors = subarray.arraySlice(bandAxis, 2, 2 + order + 1)
    response = subarray.arraySlice(bandAxis, 0, 1)
    coeff = predictors.matrixSolve(response)
    coeff = coeff.arrayProject([0]).arrayFlatten(coeffFlattener)
    return coeff

In [13]:
#对于其余部分，使用modis_res作为图像列表
evi_res = evi_res.toList(evi_res.size())
runLength = ee.List.sequence(0, evi_res.size().subtract(window_size))

In [14]:
#对该级数运行SG求解器，并返回平滑后的图像版本
def func_whq(i):
    ref = ee.Image(evi_res.get(ee.Number(i).add(half_window)))
    return getLocalFit(i).multiply(ref.select(indepSelectors)).reduce(ee.Reducer.sum()).copyProperties(ref)

sg_series = runLength.map(func_whq)

In [15]:
#添加图像层到地图并显示它。
Map.addLayer(evi_collections, {'min': -1, 'max': 1}, 'evi')
Map.addLayer(mndwi_collections, {'min': -1, 'max': 1}, 'mndwi')
#Map.addLayer(sg_series), {'min': -1, 'max': 1}, 'sg_evi')